# Homework 4 : MapReduce and Deep Learning (100 points)

*------------

** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

Required reading:
* [Reinforcement learning](http://karpathy.github.io/2016/05/31/rl/)


# Problem 1 (50 points): Using MapReduce framework to implement Matrix Multiplication
In this question, you need to use your editor to work on your code, instead of iPython notebook.
In the folder, we have a file **input_matrices.data** for the input matrices A and B. 
The goal is to create a file, named **mr_matrix_multiplication.py**, to compute the matrix C = A X B.
In this file, you should use the MapReduce framework to solve the problem, by implementing a *map* function and *reduce* function.

Please run your code through the terminal and save the output results into a file named **output**.
For example, in linux/mac, you could use the following command in the terminal:

 *cat input_matrices.data | python mr_matrix_multiplication.py > output*
 
 Note in this question, you cannot use any matrix multiplication function in existing packages, such as those in numpy and scipy. The output file should use a format that is similar to the format of the file *input_matrices.data*. Each line of the file represents one element of Matrix C. 
 For example, any of the following formats would be good.
* C, 1, 1, 132 
* "C", 1, 1, 132 
* "C" (1, 1, 132)
* _ ("C", 1, 1, 132)

Here 132 is just a made-up number, which may not be the correct answer for C[1,1].
We assume the indices of the matrix elements start from 1, instead of 0. 


## Problem 2 (50 points): Reinforcement Learning for Pacman
We first introduce an example of how to use the gym package from OpenAI to design an agent for Pacman Game.
In the following cell, we implemented a simple agent, which randomly picks the next action without looking at the screen image (i.e., the *observation*) or the reward.

Change the following code for **myAgent** to design a better agent, which takes the observation and reward as the input and picks the best action as the next move.
The agent should be able to improve itself after playing more games.

***The goals***: Implement an agent using neural networks that can achieve all the following goals:
* (a) move the PacMan in all directions.  (10 points)
* (b) using neural network to decide what is the best next move. (20 points)
* (c) after playing each episode of the game, the agent should be able to improve itself using the experience. (20 points)

Action Code:
* 1 - UP
* 2 - RIGHT
* 3 - LEFT
* 4 - DOWN

In [2]:
up = 1
right = 2
left = 3
down = 4

In [3]:
""" Trains an agent with (stochastic) Policy Gradients on PacMan. Uses OpenAI Gym. """
import numpy as np
import cPickle as pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False# resume from previous checkpoint?
render = False # whether to render the game. You should turn this off to speed up the program.

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    # W1.shape = (hidden_neurons, inputs)
    model['W_hidden'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    # output_neurons.shape = (hidden_neurons)
    model['W_up'] = np.random.randn(H) / np.sqrt(H)
    model['W_down'] = np.random.randn(H) / np.sqrt(H)
    model['W_left'] = np.random.randn(H) / np.sqrt(H)
    model['W_right'] = np.random.randn(H) / np.sqrt(H)
    
grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    #crop the screen
    I = np.reshape(I[0:160], (160,160,3))
    I = I[:,:,1]
    I[I==111]=0
    I[I==28]=1
    I[I>1]=2
    I=I[::2,::2]
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W_hidden'], x)
    h[h<0] = 0 # ReLU nonlinearity
    p_up = sigmoid(np.dot(model['W_up'], h))
    p_down = sigmoid(np.dot(model['W_down'], h))
    p_left = sigmoid(np.dot(model['W_left'], h))
    p_right = sigmoid(np.dot(model['W_right'], h))
    total_p = p_up + p_down + p_left + p_right
    p_up /= total_p
    p_down /= total_p
    p_left /= total_p
    p_right /= total_p
    return [p_up, p_down, p_left, p_right], h # return probability of taking action 2, and hidden state

def probabilities_to_action(probabilities):
    random = np.random.uniform()
    if random < probabilities[0]:
        return up
    elif random < sum(probabilities[0:2]):
        return down
    elif random < sum(probabilities[0:3]):
        return left
    else: # random < sum(probabilities[0:4]):
        return right

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW_up = np.dot(eph.T, epdlogp[:,0]).ravel() # ravel flaten the matrix into 1-d vector
    dW_down = np.dot(eph.T, epdlogp[:,1]).ravel() # ravel flaten the matrix into 1-d vector
    dW_left = np.dot(eph.T, epdlogp[:,2]).ravel() # ravel flaten the matrix into 1-d vector
    dW_right = np.dot(eph.T, epdlogp[:,3]).ravel() # ravel flaten the matrix into 1-d vector
    print dW_up
    dh_up = np.outer(epdlogp[:, 0], model['W_up'])
    dh_up[eph <= 0] = 0 # backpro prelu
    dW_hidden = np.dot(dh_up.T, epx)
    dh_down = np.outer(epdlogp[:, 0], model['W_down'])
    dh_down[eph <= 0] = 0 # backpro prelu
    dW_hidden += np.dot(dh_down.T, epx)
    dh_left = np.outer(epdlogp[:, 0], model['W_left'])
    dh_left[eph <= 0] = 0 # backpro prelu
    dW_hidden = np.dot(dh_left.T, epx)
    dh_right = np.outer(epdlogp[:, 0], model['W_right'])
    dh_right[eph <= 0] = 0 # backpro prelu
    dW_hidden += np.dot(dh_right.T, epx)

    return {'W_hidden':dW_hidden, 'W_up':dW_up, 'W_down': dW_down, 'W_left': dW_left, 'W_right': dW_right}

env = gym.make("MsPacman-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame

# xs are board states, hs are hidden neurons over time,
# dlogps = [ [label_up, label_down...], ...]
# drs = [reward, ...]
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
    if render: env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    probabilities, h = policy_forward(x)
    
    action = probabilities_to_action(probabilities)

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    labels = np.array([action == up, action == down, action == left, action == right]) - np.array(probabilities)
    dlogps.append(labels) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward
    #print "reward:%f" % reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

        # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

        # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.iteritems():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

        # boring book-keeping
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
        print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
        if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
        reward_sum = 0
        observation = env.reset() # reset env
        prev_x = None



[2016-11-28 12:07:45,268] Making new env: MsPacman-v0


[ 0.23555067 -0.52892515  0.04622637  0.06644255  0.4893885  -0.21786299
  0.51827246 -0.74510926 -0.64666406 -0.55194508  0.10331666 -0.38398356
 -1.1155393   0.77461355  0.16881577  0.17468138  0.17127192 -0.08063927
  0.18122813 -0.18760231  0.36394204  0.72068462 -0.64955361  0.36055983
  0.34656904 -0.16239621 -0.41108046 -0.35363563  0.2720348   0.88434355
 -0.91936528 -0.32042209 -0.19193076 -0.14533906  0.0092817   0.59916981
 -0.17376264 -0.23022782 -1.12475335 -0.24116079  0.25629914  0.68494555
 -0.02133206  0.72409059  0.48305808 -0.97074328 -0.07848461 -0.04440541
 -0.57985543  0.02487741 -0.26313218  0.4939594  -0.5244822   0.60255651
  0.28271841 -0.15355441  0.76851258  0.51783137 -0.57699956 -0.38264111
 -0.92104838  0.509549   -0.22035628  0.57875807  0.68203606  0.17254183
 -0.40066786 -0.10391948 -0.00962682  0.57574893 -0.69275117  0.30442946
 -0.06236149 -1.27758099 -0.03902068 -0.15475187 -0.28618196  0.68710144
 -0.0400176  -0.2554607  -0.4253331  -0.31363347 -0

KeyboardInterrupt: 

## Here we show two examples
### Example 1 (random agent)
The following agent can move the PacMan in all directions. But it's random. So it only achieved goal (a): move the paceman in all directions.


In [15]:
import gym
num_episodes = 1 # how many episodes to play
render = True # whether to render the game. You should turn this off to speed up the program.

class myAgent(object):
  def __init__(self,env):
    self.env = env
    pass      
  def pick_next_action(self, observation, reward):
    """ observation is the current screen image. reward is the current reward in the time step """
    best_action = self.env.action_space.sample() # RANDOMLY pick an action for the next move. 
    return best_action

        
env = gym.make('MsPacman-v0') # create the game envirement
agent = myAgent(env)# create an agent

# let's play some episodes of the game
for _ in range(num_episodes): 
    observation = env.reset() # initialize the game
    episode_reward=0 # the sum of rewards in an episode
    action = env.action_space.sample() # RANDOMLY pick an action for the next move
    observation, reward, done, info = env.step(action) #execute the action and get the reward and next observation
    while not done: 
        if render: 
            env.render() # render the game
        action = agent.pick_next_action(observation,reward)
        observation, reward, done, info = env.step(action)
        episode_reward += reward #adding up the reward in the episode
        if done: # the episode is done
            print("Episode reward:{}".format(episode_reward))
            episode_reward=0

[2016-11-28 09:04:27,125] Making new env: MsPacman-v0


Episode reward:220.0


### Example 2 (left-right agent)
The following agent trains a neural network to control the PacMan. But it can only move left and right. 
So it only achieved goals (b) and (c). 

In [ ]:
""" Trains an agent with (stochastic) Policy Gradients on PacMan. Uses OpenAI Gym. """
import numpy as np
import cPickle as pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False# resume from previous checkpoint?
render = True # whether to render the game. You should turn this off to speed up the program.

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
  model = pickle.load(open('save.p', 'rb'))
else:
  model = {}
  model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
  model[''] = np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.iteritems() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.iteritems() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  #crop the screen
  I = np.reshape(I[0:160], (160,160,3))
  I = I[:,:,1]
  I[I==111]=0
  I[I==28]=1
  I[I>1]=2
  I=I[::2,::2]
  return I.astype(np.float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(xrange(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model[''], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  d = np.dot(eph.T, epdlogp).ravel() # ravel flaten the matrix into 1-d vector
  dh = np.outer(epdlogp, model[''])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, '':d}

env = gym.make("MsPacman-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
  if render: env.render()

  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability
  aprob, h = policy_forward(x)
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"
  dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

  # step the environment and get new measurements
  observation, reward, done, info = env.step(action)
  reward_sum += reward
  #print "reward:%f" % reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    eph = np.vstack(hs)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)

    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.iteritems():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None


[2016-11-28 09:04:41,706] Making new env: MsPacman-v0


resetting env. episode reward total was 80.000000. running mean: 80.000000
resetting env. episode reward total was 110.000000. running mean: 80.300000


## Example code for  processing of the screen image

In [ ]:
import gym
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

env=gym.make('MsPacman-v0')

observation = env.reset()
observation, reward, done, info = env.step(1)

# plot the current screen
plt.figure(1)
plt.imshow(np.reshape(observation, (210,160,3) ))

#crop the screen
x = np.reshape(observation[0:171], (171,160,3) )
plt.figure(2)
plt.imshow(x)


# remove background
x = x[:,:,1]
x[x==111]=0
x[x==28]=1
x[x>1]=2
# downsample
x=x[::2,::2]
plt.figure(3)
plt.imshow(x,cmap='Greys')


*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.

* **Python Code File**: filename: mr_matrix_multiplication.py
* **Output File**: results for matrix C. Filename: output


** How to submit: **
  Please submit your files through myWPI, in the Assignment "Homework 4".